In [1]:
# This is only needed on the software development machine. If you are a user, you can skip this step.
import sys
sys.path.append("C:/Users/sarwj/OneDrive - Cardiff University/Documents/GitHub/topologicpy/src")

In [2]:
# Make sure you have pip installed neo4j
from topologicpy.Vertex import Vertex
from topologicpy.Edge import Edge
from topologicpy.Wire import Wire
from topologicpy.Face import Face
from topologicpy.Shell import Shell
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex
from topologicpy.Cluster import Cluster
from topologicpy.Topology import Topology
from topologicpy.Aperture import Aperture
from topologicpy.Graph import Graph
from topologicpy.Dictionary import Dictionary
from topologicpy.Helper import Helper
from topologicpy.Plotly import Plotly
from topologicpy.Neo4j import Neo4j
from getpass import getpass
import uuid
import hashlib
import json

In [3]:
def ByOffsetArea(wire,
                area,
                offsetKey="offset",
                minOffsetKey="minOffset",
                maxOffsetKey="maxOffset",
                defaultMinOffset=0,
                defaultMaxOffset=1,
                maxIterations = 10,
                tolerance=0.0001,
                silent = False):
    """
    Creates an offset wire from the input wire based on the input area.

    Parameters
    ----------
    wire : topologic_core.Wire
        The input wire.
    area : float
        The desired area of the created wire.
    offsetKey : str , optional
        The edge dictionary key under which to store the offset value. The default is "offset".
    minOffsetKey : str , optional
        The edge dictionary key under which to find the desired minimum edge offset value. If a value cannot be found, the defaultMinOffset input parameter value is used instead. The default is "minOffset".
    maxOffsetKey : str , optional
        The edge dictionary key under which to find the desired maximum edge offset value. If a value cannot be found, the defaultMaxOffset input parameter value is used instead. The default is "maxOffset".
    defaultMinOffset : float , optional
        The desired minimum edge offset distance. The default is 0.
    defaultMaxOffset : float , optional
        The desired maximum edge offset distance. The default is 1.
    maxIterations: int , optional
        The desired maximum number of iterations to attempt to converge on a solution. The default is 10.
    silent : bool , optional
        If set to True, no error and warning messages are printed. Otherwise, they are. The default is False.
    
    Returns
    -------
    topologic_core.Wire
        The created wire.

    """
    from topologicpy.Wire import Wire
    from topologicpy.Face import Face
    from topologicpy.Topology import Topology
    from topologicpy.Dictionary import Dictionary
    import numpy as np
    from scipy.optimize import minimize
    import time

    startTime = time.time()

    def compute_offset_amounts(wire,
                               area,
                               startTime,
                               offsetKey="offset",
                               minOffsetKey="minOffset",
                               maxOffsetKey="maxOffset",
                               defaultMinOffset=0,
                               defaultMaxOffset=1,
                               eps = 0.01,
                               fTolerance = 1e-6,
                               gTolerance = 1e-6,
                               maxIterations = 10000,
                               maxTime = 10,
                               tolerance=0.0001):
        
        initial_offsets = []
        bounds = []
        for edge in edges:
            d = Topology.Dictionary(edge)
            minOffset = Dictionary.ValueAtKey(d, minOffsetKey) or defaultMinOffset
            maxOffset = Dictionary.ValueAtKey(d, maxOffsetKey) or defaultMaxOffset
            # Initial guess: small negative offsets to shrink the polygon, within the constraints
            initial_offsets.append((minOffset + maxOffset) / 2)
            # Bounds based on the constraints for each edge
            bounds.append((minOffset, maxOffset))

        # Convert initial_offsets to np.array for efficiency
        initial_offsets = np.array(initial_offsets)
        iteration_count = [0]  # List to act as a mutable counter

        def objective_function(offsets, startTime, maxTime):
            for i, edge in enumerate(edges):
                d = Topology.Dictionary(edge)
                d = Dictionary.SetValueAtKey(d, offsetKey, offsets[i])
                edge = Topology.SetDictionary(edge, d)
            
            # Offset the wire
            new_wire = Wire.ByOffset(wire, offsetKey=offsetKey, silent=silent)
            # Check for an illegal wire. In that case, return a very large loss value.
            if not Topology.IsInstance(new_wire, "Wire"):
                return (float("inf"))
            if not Wire.IsManifold(new_wire):
                return (float("inf"))
            if not Wire.IsClosed(new_wire):
                return (float("inf"))
            new_face = Face.ByWire(new_wire)
            # Calculate the area of the new wire/face
            new_area = Face.Area(new_face)
            
            # The objective is the difference between the target hole area and the actual hole area
            # We want this difference to be as close to 0 as possible
            loss = (new_area - area) ** 2
            # If the loss is less than the tolerance, accept the result and return a loss of 0.
            if loss < tolerance:
                return 0
            # Otherwise, return the actual loss value.
            return loss 
        
        # Callback function to track and display iteration number
        def iteration_callback(xk):
            iteration_count[0] += 1  # Increment the counter
            if not silent:
                print(f"Wire.ByOffsetArea - Information: Iteration {iteration_count[0]}")
        
        # Use scipy optimization/minimize to find the correct offsets, respecting the min/max bounds
        result = minimize(objective_function,
                        initial_offsets,
                        args = (startTime, maxTime),
                        #method='L-BFGS-B',
                        method = "Powell",
                        bounds=bounds,
                        options={ 'maxiter': maxIterations},
                        callback=iteration_callback
                        )

        # Return the offsets
        return result.x
    
    if not Topology.IsInstance(wire, "Wire"):
        if not silent:
            print("Wire.OffsetByArea - Error: The input wire parameter is not a valid wire. Returning None.")
        return None
    
    if not Wire.IsManifold(wire):
        if not silent:
            print("Wire.OffsetByArea - Error: The input wire parameter is not a manifold wire. Returning None.")
        return None
    
    if not Wire.IsClosed(wire):
        if not silent:
            print("Wire.OffsetByArea - Error: The input wire parameter is not a closed wire. Returning None.")
        return None
    
    edges = Topology.Edges(wire)
    # Compute the offset amounts
    offsets = compute_offset_amounts(wire,
                            area = area,
                            startTime = startTime,
                            offsetKey = offsetKey,
                            minOffsetKey = minOffsetKey,
                            maxOffsetKey = maxOffsetKey,
                            defaultMinOffset = defaultMinOffset,
                            defaultMaxOffset = defaultMaxOffset,
                            maxIterations = maxIterations,
                            tolerance = tolerance)
    # Set the edge dictionaries correctly according to the specified offsetKey
    for i, edge in enumerate(edges):
        d = Topology.Dictionary(edge)
        d = Dictionary.SetValueAtKey(d, offsetKey, offsets[i])
        edge = Topology.SetDictionary(edge, d)
            
    # Offset the wire
    return_wire = Wire.ByOffset(wire, offsetKey=offsetKey, silent=silent)
    if not Topology.IsInstance(wire, "Wire"):
        if not silent:
            print("Wire.OffsetByArea - Error: Could not create the offset wire. Returning None.")
        return None
    return return_wire


# Create the wire
#wire = Wire.Rectangle(width=4, length=4)
wire = Wire.Einstein(radius=4)
face = Face.ByWire(wire)
print("Original Area:", Face.Area(face))
edges = Topology.Edges(wire)
for i, edge in enumerate(edges):
    if i%2 == 0:
        d = Dictionary.ByKeysValues(["minOffset", "maxOffset"], [0.05,1])
    else:
        d = Dictionary.ByKeysValues(["minOffset", "maxOffset"], [0.05,0.5])
    edge = Topology.SetDictionary(edge, d)


area = 40.0  # Desired hole area

hole = Wire.ByOffsetArea(wire,
                    area,
                    offsetKey="offset",
                    minOffsetKey="minOffset",
                    maxOffsetKey="maxOffset",
                    defaultMinOffset=0,
                    defaultMaxOffset=1,
                    maxIterations = 2,
                    tolerance=0.01,
                    silent = False)

if Topology.IsInstance(hole, "wire"):
    if Wire.IsManifold(hole) and Wire.IsClosed(hole):
        hole_face = Face.ByWire(hole)
        print("Hole Area:", Face.Area(hole_face))
        Topology.Show(wire, hole_face)
else:
    print("Operation Failed!")

Original Area: 55.425626
Wire.ByOffsetArea - Information: Iteration 1
Wire.ByOffsetArea - Information: Iteration 2


NameError: name 'edges' is not defined

In [ ]:
Topology.Show(hole)